In [73]:
# i want one clean pass that only renames and exports (no dropping)
from pathlib import Path
import pandas as pd
import re

OVERWRITE = False  # if files already exist, skip them. set True only if i need to redo.

base = Path(r"C:\Users\venus\OneDrive\Documents\GitHub\Climate-Malnutrition-EastAfrica")
unzipped = base / "data_raw" / "unzipped"          # where my renamed .dta files live
std_dir  = base / "data_cleaned" / "standardized"  # parquet outputs
csv_dir  = std_dir / "csv_exports"                 # csv outputs

std_dir.mkdir(parents=True, exist_ok=True)
csv_dir.mkdir(parents=True, exist_ok=True)


In [75]:
# i’m only renaming common codes to readable names
rename_map = {
    "caseid": "case_id",
    "v000": "survey_code",
    "v001": "cluster",
    "v002": "household",
    "v003": "mother_line",
    "v004": "sample_strata",
    "v005": "weight_raw",
    "v006": "month_interview",
    "v007": "year_interview",
    "v008": "century_month_code",
    "b4": "child_sex",
    "b5": "child_alive",
    "hw1": "child_age_months",
    "hw2": "child_weight_kg",
    "hw3": "child_height_cm",
    "hw70": "z_height_for_age",
    "hw71": "z_weight_for_height",
    "hw72": "z_weight_for_age",
}


In [77]:
def list_dta_files():
    # look for all .dta files I already renamed like Ethiopia_KR_2016.dta, etc.
    return sorted(unzipped.rglob("*.dta"))

def ensure_unique_columns(cols):
    # if a column name repeats, i’ll add a suffix so nothing gets dropped
    seen = {}
    out = []
    for c in cols:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}__dup{seen[c]}")
    return out

def parse_country_year_from_name(name):
    # pulls country + year token from filename like Ethiopia_KR_2016.dta
    m = re.match(r"([A-Za-z]+)_KR_([0-9A-Za-z]+)\.dta$", name)
    if m:
        return m.group(1), m.group(2)
    return "Unknown", "Unknown"


In [81]:
dta_files = list_dta_files()
print("found .dta files:", len(dta_files))

for dta in dta_files:
    base_name = dta.name.replace(".dta", "_cleaned")
    pq_path  = std_dir / f"{base_name}.parquet"
    csv_path = csv_dir  / f"{base_name}.csv"

    # skip if both outputs already exist and i don’t want to overwrite
    if not OVERWRITE and pq_path.exists() and csv_path.exists():
        print("skip existing:", dta.name)
        continue

    # read the stata file with raw codes; i’m not converting categories
    df = pd.read_stata(dta, convert_categoricals=False, preserve_dtypes=False)

    # first check duplicates before rename
if len(df.columns) != len(set(df.columns)):
    df.columns = ensure_unique_columns(df.columns)

# rename the common DHS variable codes
df = df.rename(columns=rename_map)

# re-check duplicates after renaming (some names like caseid → case_id can collide)
if len(df.columns) != len(set(df.columns)):
    df.columns = ensure_unique_columns(df.columns)


    # label inside the file so i always know where each row came from
    country, year_token = parse_country_year_from_name(dta.name)
    df["file_country"] = country
    df["file_year"] = year_token
    df["file_type"] = "KR"
    df["original_filename"] = dta.name

    # write both formats with the exact same basename
    df.to_parquet(pq_path, index=False)
    df.to_csv(csv_path, index=False)

    print("exported:", dta.name, "→", pq_path.name, "and", csv_path.name)


found .dta files: 25
skip existing: Ethiopia_KR_2005.dta
skip existing: Ethiopia_KR_51.dta
skip existing: Ethiopia_KR_2011.dta
skip existing: Ethiopia_KR_2016.dta
skip existing: Ethiopia_KR_2021.dta


In [83]:
inputs = [p.name.replace(".dta","_cleaned") for p in list_dta_files()]
parquets = [p.name.replace(".parquet","") for p in std_dir.glob("*_cleaned.parquet")]
csvs     = [p.name.replace(".csv","") for p in csv_dir.glob("*_cleaned.csv")]

print("inputs:", len(inputs))
print("parquets:", len(parquets))
print("csvs:", len(csvs))
print("missing parquet for:", sorted(set(inputs) - set(parquets)))
print("missing csv for:", sorted(set(inputs) - set(csvs)))


inputs: 25
parquets: 5
csvs: 5
missing parquet for: ['Kenya_KR_1988_cleaned', 'Kenya_KR_1998_cleaned', 'Kenya_KR_1999_cleaned', 'Kenya_KR_2008_cleaned', 'Kenya_KR_2010_cleaned', 'Kenya_KR_2022_cleaned', 'Kenya_KR_72_cleaned', 'Tanzania_KR_1996_cleaned', 'Tanzania_KR_1999_cleaned', 'Tanzania_KR_2005_cleaned', 'Tanzania_KR_2007_cleaned', 'Tanzania_KR_2013_cleaned', 'Tanzania_KR_2018_cleaned', 'Tanzania_KR_2023_cleaned', 'Uganda_KR_1988_cleaned', 'Uganda_KR_2000_cleaned', 'Uganda_KR_2005_cleaned', 'Uganda_KR_2010_cleaned', 'Uganda_KR_2011_cleaned', 'Uganda_KR_2018_cleaned']
missing csv for: ['Kenya_KR_1988_cleaned', 'Kenya_KR_1998_cleaned', 'Kenya_KR_1999_cleaned', 'Kenya_KR_2008_cleaned', 'Kenya_KR_2010_cleaned', 'Kenya_KR_2022_cleaned', 'Kenya_KR_72_cleaned', 'Tanzania_KR_1996_cleaned', 'Tanzania_KR_1999_cleaned', 'Tanzania_KR_2005_cleaned', 'Tanzania_KR_2007_cleaned', 'Tanzania_KR_2013_cleaned', 'Tanzania_KR_2018_cleaned', 'Tanzania_KR_2023_cleaned', 'Uganda_KR_1988_cleaned', 'Uganda_K

In [85]:
def ensure_unique_columns(cols):
    # if a column name repeats, i suffix it so nothing is dropped
    seen = {}
    out = []
    for c in map(str, cols):  # make sure everything is a string
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}__dup{seen[c]}")
    return out


In [87]:
from pathlib import Path

# inputs from .dta
inputs = [p for p in list_dta_files()]
input_bases = [p.name.replace(".dta","_cleaned") for p in inputs]

# already-produced outputs
parquets = [p.name.replace(".parquet","") for p in std_dir.glob("*_cleaned.parquet")]
csvs     = [p.name.replace(".csv","") for p in csv_dir.glob("*_cleaned.csv")]

have_both = set(parquets).intersection(set(csvs))
missing_bases = [b for b in input_bases if b not in have_both]

print("total inputs:", len(inputs))
print("already done:", len(have_both))
print("still missing:", len(missing_bases))
missing_bases[:10]  # preview


total inputs: 25
already done: 5
still missing: 20


['Kenya_KR_1988_cleaned',
 'Kenya_KR_1998_cleaned',
 'Kenya_KR_1999_cleaned',
 'Kenya_KR_2008_cleaned',
 'Kenya_KR_2010_cleaned',
 'Kenya_KR_72_cleaned',
 'Kenya_KR_2022_cleaned',
 'Tanzania_KR_1996_cleaned',
 'Tanzania_KR_1999_cleaned',
 'Tanzania_KR_2005_cleaned']

In [89]:
import pandas as pd
import re

def parse_country_year_from_name(name):
    m = re.match(r"([A-Za-z]+)_KR_([0-9A-Za-z]+)\.dta$", name)
    if m:
        return m.group(1), m.group(2)
    return "Unknown", "Unknown"

log = []  # collect status per file

for dta in inputs:
    base_clean = dta.name.replace(".dta","_cleaned")
    if base_clean not in missing_bases:
        # already produced both parquet and csv — skip
        continue

    pq_path  = std_dir / f"{base_clean}.parquet"
    csv_path = csv_dir  / f"{base_clean}.csv"

    try:
        # read raw; do not convert categories
        df = pd.read_stata(dta, convert_categoricals=False, preserve_dtypes=False)

        # ensure unique BEFORE rename
        if len(df.columns) != len(set(df.columns)):
            df.columns = ensure_unique_columns(df.columns)

        # apply your rename map (rename-only, no drops)
        df = df.rename(columns=rename_map)

        # ensure unique AFTER rename (caseid -> case_id can collide)
        if len(df.columns) != len(set(df.columns)):
            df.columns = ensure_unique_columns(df.columns)

        # label inside
        country, year_token = parse_country_year_from_name(dta.name)
        df["file_country"] = country
        df["file_year"] = year_token
        df["file_type"] = "KR"
        df["original_filename"] = dta.name

        # write both formats with identical basename
        df.to_parquet(pq_path, index=False)
        df.to_csv(csv_path, index=False)

        print("exported:", dta.name, "→", pq_path.name, "and", csv_path.name)
        log.append({"file": dta.name, "status": "ok", "rows": len(df)})

    except Exception as e:
        print("FAILED:", dta.name, "reason:", str(e))
        log.append({"file": dta.name, "status": "fail", "reason": str(e)})


exported: Kenya_KR_1988.dta → Kenya_KR_1988_cleaned.parquet and Kenya_KR_1988_cleaned.csv
exported: Kenya_KR_1998.dta → Kenya_KR_1998_cleaned.parquet and Kenya_KR_1998_cleaned.csv
exported: Kenya_KR_1999.dta → Kenya_KR_1999_cleaned.parquet and Kenya_KR_1999_cleaned.csv
exported: Kenya_KR_2008.dta → Kenya_KR_2008_cleaned.parquet and Kenya_KR_2008_cleaned.csv
exported: Kenya_KR_2010.dta → Kenya_KR_2010_cleaned.parquet and Kenya_KR_2010_cleaned.csv
exported: Kenya_KR_72.dta → Kenya_KR_72_cleaned.parquet and Kenya_KR_72_cleaned.csv
exported: Kenya_KR_2022.dta → Kenya_KR_2022_cleaned.parquet and Kenya_KR_2022_cleaned.csv
exported: Tanzania_KR_1996.dta → Tanzania_KR_1996_cleaned.parquet and Tanzania_KR_1996_cleaned.csv
exported: Tanzania_KR_1999.dta → Tanzania_KR_1999_cleaned.parquet and Tanzania_KR_1999_cleaned.csv
exported: Tanzania_KR_2005.dta → Tanzania_KR_2005_cleaned.parquet and Tanzania_KR_2005_cleaned.csv
exported: Tanzania_KR_2007.dta → Tanzania_KR_2007_cleaned.parquet and Tanzania_

In [91]:
inputs = [p.name.replace(".dta","_cleaned") for p in list_dta_files()]
parquets = [p.name.replace(".parquet","") for p in std_dir.glob("*_cleaned.parquet")]
csvs     = [p.name.replace(".csv","") for p in csv_dir.glob("*_cleaned.csv")]

print("inputs:", len(inputs))
print("parquets:", len(parquets))
print("csvs:", len(csvs))
print("missing parquet for:", sorted(set(inputs) - set(parquets)))
print("missing csv for:", sorted(set(inputs) - set(csvs)))


inputs: 25
parquets: 25
csvs: 25
missing parquet for: []
missing csv for: []
